# AuxTel Calibration Illumination System monochromator slit test.

Craig Lage - 13Feb23 - modified 01Dec23

In [ ]:
import asyncio
import sys, time, os
from lsst.ts import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time, TimeDelta
from astropy.table import Table

from lsst.ts import salobj

In [ ]:
# change this to a local directory
data_dir = '/home/craiglagegit/DATA'

In [ ]:
d = salobj.Domain()

### Connect to CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

In [ ]:
await atmonochromator.start_task
await FiberSpectrograph.start_task
await electrometer.start_task
await WhiteLightSource.start_task

In [ ]:
# Check that the monochromator has grating=1 and lambda=500
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)


# functions to run the electromter measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    return filename, lfa, data

async def fiber_meas(exp_time):
    FiberSpectrograph.evt_largeFileObjectAvailable.flush()
    tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
    lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    spec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {spec_filen}")
    hdu = fits.open(f'{spec_filen}')
    wavelength = hdu[1].data['wavelength'].flatten()
    spectrum = hdu[0].data
    return filename, lfa, wavelength, spectrum


In [ ]:
# Run electrometer calibration
#await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)    

### Test Electrometer

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

### Set up Monochromator

In [ ]:
wave=500
entry_width=0.5
exit_width=0.5
grating = 1
await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=wave, \
                                                             gratingType=grating, \
                                                             fontEntranceSlitWidth=entry_width, \
                                                             fontExitSlitWidth=exit_width, timeout=180)

In [ ]:
await get_params()

In [ ]:
# Now run a matrix of slit widths
results = {}
slit_2 = 0.5
for i in range(1, 11):
    slit_1 = 0.5 * i
    for j in range(1, 11):
        exp_time = 1.0
        slit_2 = 0.5 * j
        key = f"{slit_1:.1f}_{slit_2:.1f}"
        await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=wave, \
                                                             gratingType=grating, \
                                                             fontEntranceSlitWidth=slit_1, \
                                                             fontExitSlitWidth=slit_2, timeout=180)  
        #filename, electrometer_lfa, electrometer_data = await elec_meas(1.0)
        #for (t,current) in electrometer_data:
        #    electrometer_current = current
        filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
        spec_max = np.max(spectrum)
        while spec_max > 15900:
            exp_time /= 2.0
            filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
            spec_max = np.max(spectrum)
            
        #print(electrometer_current, exp_time, spec_max)
        results[key] = {}
        results[key]['electrometer_lfa_url']=None#electrometer_lfa.url
        results[key]['fiber_lfa_url']=fiber_lfa.url
        results[key]['electrometer_current']=None#electrometer_current
        results[key]['fiber_data']=[wavelengths, spectrum, exp_time]
        print(f"Done with {slit_1}, {slit_2}")

In [ ]:
# Pickle the data
import pickle as pkl
filename = "/home/craiglagegit/DATA/monochromator_slit_matrix_01dec23.pkl"
file = open(filename, "wb")
pkl.dump(results, file)
file.close()

## Structure of the pickle file:
key = '0.5_0.5' where the slit widths run from 0.5 to 5.0 in steps of 0.5 \
results[key]['electrometer_lfa_url']=electrometer_lfa.url \
results[key]['fiber_lfa_url']=fiber_lfa.url \
results[key]['electrometer_current']=electrometer_current \
results[key]['fiber_data']=[wavelengths, spectrum, exp_time] 


In [ ]:
#Look at one of the spectra

slit_1 = 0.5
slit_2 = 0.5
key = f"{slit_1:.1f}_{slit_2:.1f}"
line_min = 450.0
line_max = 550.0
[wavelengths, spectrum, exp_time] = results[key]['fiber_data']
spectrum_sum = 0.0
for i, wl in enumerate(wavelengths):
    if wl > line_min and wl < line_max:
        spectrum_sum += spectrum[i]
spectrum_sum /= exp_time # Account for different exposure times
print(exp_time, spectrum_sum)
plt.plot(wavelengths, spectrum)

In [ ]:
# Now plot the results
%matplotlib inline
slit_1s = np.arange(0.5, 5.5, 0.5)
slit_2s = np.arange(0.5, 5.5, 0.5)
Z = np.zeros([10,10])
[X,Y] = np.meshgrid(slit_1s, slit_2s);
line_min = 450.0
line_max = 550.0
    
for i, s1 in enumerate(slit_1s):
    for j, s2 in enumerate(slit_2s):
        # Add up the spectrum counts around the 500 nm line
        # from 450nm to 550nm
        key = f"{s1:.1f}_{s2:.1f}"
        [wavelengths, spectrum, exp_time] = results[key]['fiber_data']
        spectrum_sum = 0.0
        for ii, wl in enumerate(wavelengths):
            if wl > line_min and wl < line_max:
                spectrum_sum += spectrum[ii]
        spectrum_sum /= exp_time # Account for different exposure times
        Z[i,j] = np.log10(spectrum_sum)

img=plt.contourf(X,Y,Z)
plt.colorbar(img)                
plt.xlabel("Slit 1 width (mm)")
plt.ylabel("Slit 2 width (mm)")
plt.title("Log10(Spectral line sum) vs slit widths\n wavelength=500, grating 0")
plt.savefig(f"/home/craiglagegit/DATA/Monochromator_Slit_Test_13Apr23.png")